In [2]:
import pandas as pd
import torch
import pickle

In [3]:
import sys
sys.path.insert(0, "../src")   
from model_utils import load_models_from_dir
from model_utils import predict_next_days_for_sids, load_scalers

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
test_df = pd.read_excel(
        "../output/data/2025_05_30_test_top_10_store_10_store_item.xlsx",
)
test_df = test_df.sort_values(by=['date', 'store_item'])
test_df = test_df.reset_index(drop=True)
test_df.head(5)


,id,date,store_item,store_nbr,item_nbr,onpromotion,unit_sales
0,125506246,2017-08-16,3_1047679,3,1047679,0,0
1,125504870,2017-08-16,3_114790,3,114790,1,0
2,125505116,2017-08-16,3_305229,3,305229,0,0
3,125505140,2017-08-16,3_314384,3,314384,1,0
4,125505228,2017-08-16,3_364606,3,364606,0,0


In [6]:
test_df["date"].nunique()
test_df["date"].min(), test_df["date"].max()

(Timestamp('2017-08-16 00:00:00'), Timestamp('2017-08-31 00:00:00'))

In [7]:

def preprocess_test_df(df, feature_cols):
    df['onpromotion'] = df['onpromotion'].astype(bool).astype(int)
    df[feature_cols] = df[feature_cols].astype('float32')
    df["date"] = pd.to_datetime(df["date"])
    df.sort_values("date", inplace=True)
    df.reset_index(drop=True, inplace=True)
    df["store_item"] = df["store_item"].astype(str)
    df.rename(columns={"store_nbr": "store", "item_nbr": "item"}, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df




In [8]:
test_df = preprocess_test_df(test_df, ["unit_sales"])
test_df.dtypes

id                      int64
date           datetime64[ns]
store_item             object
store                   int64
item                    int64
onpromotion             int64
unit_sales            float32
dtype: object

In [9]:
test_df.head(5)

,id,date,store_item,store,item,onpromotion,unit_sales
0,125506246,2017-08-16,3_1047679,3,1047679,0,0.0
1,125688463,2017-08-16,50_305229,50,305229,0,0.0
2,125688217,2017-08-16,50_114790,50,114790,1,0.0
3,125689593,2017-08-16,50_1047679,50,1047679,0,0.0
4,125684994,2017-08-16,49_584028,49,584028,0,0.0


In [10]:
test_df.query("store_item == '3_1047679'")

,id,date,store_item,store,item,onpromotion,unit_sales
0,125506246,2017-08-16,3_1047679,3,1047679,0,0.0
175,125716900,2017-08-17,3_1047679,3,1047679,0,0.0
271,125927554,2017-08-18,3_1047679,3,1047679,0,0.0
372,126138208,2017-08-19,3_1047679,3,1047679,0,0.0
472,126348862,2017-08-20,3_1047679,3,1047679,0,0.0
572,126559516,2017-08-21,3_1047679,3,1047679,0,0.0
671,126770170,2017-08-22,3_1047679,3,1047679,0,0.0
772,126980824,2017-08-23,3_1047679,3,1047679,0,0.0
872,127191478,2017-08-24,3_1047679,3,1047679,0,0.0
972,127402132,2017-08-25,3_1047679,3,1047679,0,0.0


In [11]:
df = pd.read_excel("../output/data/20250530_train_top_10_store_10_item_sales_cyclical_features_16_days_X_y.xlsx")
df.head(5)

,start_date,store_item,store,item,dayofweek_sin_1,dayofweek_cos_1,weekofmonth_sin_1,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,...,y_item_med_day_7,y_item_med_day_8,y_item_med_day_9,y_item_med_day_10,y_item_med_day_11,y_item_med_day_12,y_item_med_day_13,y_item_med_day_14,y_item_med_day_15,y_item_med_day_16
0,2013-01-09,3_1047679,3,1047679,0.974928,-0.222521,0.587785,-0.809017,0.500000,8.660254e-01,...,62.5,89.5,148.0,142.5,102.0,76.0,79.0,70.0,79.5,95.0
1,2013-01-25,3_1047679,3,1047679,-0.433884,-0.900969,-0.951057,0.309017,0.500000,8.660254e-01,...,74.5,84.5,93.0,65.5,70.0,41.0,69.0,76.0,151.0,57.0
2,2013-02-10,3_1047679,3,1047679,-0.781831,0.623490,0.587785,-0.809017,0.866025,5.000000e-01,...,55.5,63.0,84.0,53.5,114.5,118.0,156.0,50.0,54.0,132.0
3,2013-02-26,3_1047679,3,1047679,0.781831,0.623490,-0.951057,0.309017,0.866025,5.000000e-01,...,80.5,51.0,68.5,135.5,157.0,89.0,30.0,62.0,86.0,107.0
4,2013-03-14,3_1047679,3,1047679,0.433884,-0.900969,0.587785,-0.809017,1.000000,6.123234e-17,...,120.5,188.0,204.0,103.0,54.0,73.5,58.5,98.5,178.0,225.5


In [12]:
# Get the latest row per store_item
idx = df.groupby("store_item")["start_date"].idxmax()
last_date_df = df.loc[idx].reset_index(drop=True)

# Identify y_ columns
y_cols = [col for col in df.columns if col.startswith("y_")]

# Meta columns to carry over
meta_cols = ["start_date", "store_item", "store", "item"]

# Build new rows
new_rows = []

for _, row in last_date_df.iterrows():
    new_X = {col.replace("y_", "", 1): row[col] for col in y_cols}  # only first "y_" is removed
    new_y = {col: 0.0 for col in y_cols}
    meta = {col: row[col] for col in meta_cols}
    meta["start_date"] = pd.to_datetime(meta["start_date"]) + pd.Timedelta(days=16)
    new_row = {**meta, **new_X, **new_y}
    new_rows.append(new_row)

# Final DataFrame with new rows
last_date_df = pd.DataFrame(new_rows)
last_date_df


,start_date,store_item,store,item,dayofweek_sin_1,dayofweek_cos_1,weekofmonth_sin_1,weekofmonth_cos_1,monthofyear_sin_1,monthofyear_cos_1,...,y_item_med_day_7,y_item_med_day_8,y_item_med_day_9,y_item_med_day_10,y_item_med_day_11,y_item_med_day_12,y_item_med_day_13,y_item_med_day_14,y_item_med_day_15,y_item_med_day_16
0,2017-07-31,3_1047679,3,1047679,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-07-31,3_114790,3,114790,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-07-31,3_305229,3,305229,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-07-31,3_314384,3,314384,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-07-31,3_364606,3,364606,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2017-07-31,8_502331,8,502331,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,2017-07-31,8_567623,8,567623,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,2017-07-31,8_581078,8,581078,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,2017-07-31,8_582864,8,582864,0.0,1.0,-2.449294e-16,1.0,-0.5,-0.866025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
print("Unique (store, item) pairs:", last_date_df["store_item"].nunique())

Unique (store, item) pairs: 100


In [15]:
models = load_models_from_dir("../output/models/", date_str="2025-05-31")

In [19]:
x_scalers, y_scalers = load_scalers(date_str="2025-05-31")

In [21]:
# --- Extract model and feature columns ---
sid = "50_114790"
model, feature_cols = models[sid]
model.eval()

# --- Prepare input ---
input_data = last_date_df.query("store_item == @sid")
x = input_data[feature_cols].values.astype("float32")
x_tensor = torch.tensor(x)

# --- Make prediction ---
with torch.no_grad():
    y_pred = model(x_tensor)

# --- Convert prediction to DataFrame with feature column names ---
y_pred_scaled = pd.DataFrame(y_pred.numpy(), columns=feature_cols)
y_scaler = y_scalers[sid]
y_pred_df = pd.DataFrame(y_scaler.inverse_transform(y_pred_scaled), columns=feature_cols)

# --- Keep only 'sales_day_' columns ---
sales_day_cols = [col for col in y_pred_df.columns if col.startswith("sales_day_")]
sales_pred_df = y_pred_df[sales_day_cols]
sales_pred_df = np.expm1(sales_pred_df)

# --- Extract metadata ---
meta = input_data.iloc[0][["store_item", "store", "item"]].to_dict()
start_date = pd.to_datetime(input_data.iloc[0]["start_date"]) + pd.Timedelta(days=15)
# --- Build final output: one row per predicted day ---
rows = []
for i, col in enumerate(sales_day_cols):
    row = {
        "date": start_date + pd.Timedelta(days=i + 1),
        **meta,  # Add metadata first
        "unit_sales": sales_pred_df.at[0, col]  # Add 'sales' last
    }
    rows.append(row)

# --- Final DataFrame ---
final_df = pd.DataFrame(rows)
final_df


,date,store_item,store,item,unit_sales
0,2017-08-16,50_114790,50,114790,63.000000
1,2017-08-17,50_114790,50,114790,69.000023
2,2017-08-18,50_114790,50,114790,64.000015
3,2017-08-19,50_114790,50,114790,72.999977
4,2017-08-20,50_114790,50,114790,64.000015
5,2017-08-21,50_114790,50,114790,67.000000
6,2017-08-22,50_114790,50,114790,73.000008
7,2017-08-23,50_114790,50,114790,68.999992
8,2017-08-24,50_114790,50,114790,79.000000
9,2017-08-25,50_114790,50,114790,107.000008


In [22]:
final_df.dtypes

date          datetime64[ns]
store_item            object
store                  int64
item                   int64
unit_sales           float32
dtype: object

In [23]:
test_df.dtypes

id                      int64
date           datetime64[ns]
store_item             object
store                   int64
item                    int64
onpromotion             int64
unit_sales            float32
dtype: object

In [24]:

# Step 2: Merge with test_df using a left join

merged_df = test_df.query("store_item == @sid").merge(
    final_df,
    on=["date", "store_item", "store", "item"],
    how="left",
    suffixes=("", "_pred")
)

# Step 3: Replace test_df unit_sales with predicted ones where available
merged_df["unit_sales"] = merged_df["unit_sales_pred"].combine_first(merged_df["unit_sales"])

# Step 4: Drop the helper column
merged_df = merged_df.drop(columns=["unit_sales_pred"])
merged_df


,id,date,store_item,store,item,onpromotion,unit_sales
0,125688217,2017-08-16,50_114790,50,114790,1,63.000000
1,125898871,2017-08-17,50_114790,50,114790,1,69.000023
2,126109525,2017-08-18,50_114790,50,114790,1,64.000015
3,126320179,2017-08-19,50_114790,50,114790,1,72.999977
4,126530833,2017-08-20,50_114790,50,114790,1,64.000015
5,126741487,2017-08-21,50_114790,50,114790,1,67.000000
6,126952141,2017-08-22,50_114790,50,114790,0,73.000008
7,127162795,2017-08-23,50_114790,50,114790,0,68.999992
8,127373449,2017-08-24,50_114790,50,114790,0,79.000000
9,127584103,2017-08-25,50_114790,50,114790,0,107.000008


In [26]:
all_preds_df = predict_next_days_for_sids(last_date_df, models, y_scalers)
all_preds_df.head()


,date,store_item,store,item,unit_sales
0,2017-08-16,44_582864,44,582864,127.859993
1,2017-08-17,44_582864,44,582864,175.554016
2,2017-08-18,44_582864,44,582864,0.000000
3,2017-08-19,44_582864,44,582864,174.439041
4,2017-08-20,44_582864,44,582864,200.261017


In [27]:
# Step 2: Merge with test_df using a left join
merged_df = test_df.merge(
    all_preds_df,
    on=["date", "store_item", "store", "item"],
    how="left",
    suffixes=("", "_pred")
)

# Step 3: Replace test_df unit_sales with predicted ones where available
merged_df["unit_sales"] = merged_df["unit_sales_pred"].combine_first(merged_df["unit_sales"])

# Step 4: Drop the helper column
merged_df = merged_df.drop(columns=["unit_sales_pred"])
merged_df

,id,date,store_item,store,item,onpromotion,unit_sales
0,125506246,2017-08-16,3_1047679,3,1047679,0,906.000061
1,125688463,2017-08-16,50_305229,50,305229,0,64.000015
2,125688217,2017-08-16,50_114790,50,114790,1,63.000000
3,125689593,2017-08-16,50_1047679,50,1047679,0,5.000000
4,125684994,2017-08-16,49_584028,49,584028,0,153.690994
...,...,...,...,...,...,...,...
1595,128829180,2017-08-31,45_581078,45,581078,0,70.000015
1596,128829153,2017-08-31,45_567623,45,567623,0,124.999985
1597,128829055,2017-08-31,45_502331,45,502331,0,179.999954
1598,128832956,2017-08-31,46_502331,46,502331,0,175.000015


In [28]:
merged_df.query("store_item == '3_1047679'")
    

,id,date,store_item,store,item,onpromotion,unit_sales
0,125506246,2017-08-16,3_1047679,3,1047679,0,906.000061
175,125716900,2017-08-17,3_1047679,3,1047679,0,795.999695
271,125927554,2017-08-18,3_1047679,3,1047679,0,1062.999756
372,126138208,2017-08-19,3_1047679,3,1047679,0,824.000122
472,126348862,2017-08-20,3_1047679,3,1047679,0,986.000000
572,126559516,2017-08-21,3_1047679,3,1047679,0,826.000183
671,126770170,2017-08-22,3_1047679,3,1047679,0,840.000244
772,126980824,2017-08-23,3_1047679,3,1047679,0,863.999695
872,127191478,2017-08-24,3_1047679,3,1047679,0,861.000122
972,127402132,2017-08-25,3_1047679,3,1047679,0,788.000000


In [29]:
merged_df.query("store_item == '50_1047679'")


,id,date,store_item,store,item,onpromotion,unit_sales
3,125689593,2017-08-16,50_1047679,50,1047679,0,5.000000
106,125900247,2017-08-17,50_1047679,50,1047679,0,4.000000
205,126110901,2017-08-18,50_1047679,50,1047679,0,58.000004
302,126321555,2017-08-19,50_1047679,50,1047679,0,5.000000
403,126532209,2017-08-20,50_1047679,50,1047679,0,61.999992
505,126742863,2017-08-21,50_1047679,50,1047679,0,59.000004
606,126953517,2017-08-22,50_1047679,50,1047679,0,52.000008
702,127164171,2017-08-23,50_1047679,50,1047679,0,3.000000
805,127374825,2017-08-24,50_1047679,50,1047679,0,79.000000
909,127585479,2017-08-25,50_1047679,50,1047679,0,54.999992


In [30]:
merged_df.head()

,id,date,store_item,store,item,onpromotion,unit_sales
0,125506246,2017-08-16,3_1047679,3,1047679,0,906.000061
1,125688463,2017-08-16,50_305229,50,305229,0,64.000015
2,125688217,2017-08-16,50_114790,50,114790,1,63.000000
3,125689593,2017-08-16,50_1047679,50,1047679,0,5.000000
4,125684994,2017-08-16,49_584028,49,584028,0,153.690994


In [31]:
merged_df.to_excel("../output/data/20250531_merged_submission.xlsx", index=False)

In [32]:
submission_df = merged_df[["id", "unit_sales"]]
submission_df["unit_sales"] = submission_df["unit_sales"].clip(lower=0)
submission_df.to_csv('../output/data/20250531_submission.csv', float_format='%.4f', index=None)

/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_97321/1049799375.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df["unit_sales"] = submission_df["unit_sales"].clip(lower=0)
